# 🏗️ LFM2-1.2B GGUF 변환 및 양자화

HuggingFace 모델을 GGUF로 변환하고 양자화합니다.

## 지원 양자화
- Q8_0: 최고 품질
- Q5_K_M: 균형 (추천)
- Q4_K_M: 경량화

## 환경
- Colab CPU로 충분 (GPU 불필요)
- High-RAM 권장

In [ ]:
# 1. llama.cpp 빌드
!git clone https://github.com/ggerganov/llama.cpp
%cd /content/llama.cpp
!mkdir build && cd build && cmake .. && cmake --build . --config Release
%cd /content/llama.cpp
!pip install -r requirements.txt
!pip install huggingface_hub

In [ ]:
# 2. HuggingFace 로그인
from huggingface_hub import login
login()  # 수동 입력 또는 Colab Secrets

In [ ]:
# 3. 설정 (본인 ID로 변경)
MODEL_ID = "gyung/lfm2-1.2b-koen-mt-v8-rl-10k-merged"  # 변환할 모델
OUTPUT_REPO = "YOUR_ID/your-model-GGUF"                 # 출력 레포 (변경 필요)
MODEL_NAME = "lfm2-1.2b-koen-mt-v8"                     # 파일명 prefix

In [ ]:
# 4. 모델 다운로드
from huggingface_hub import snapshot_download

local_path = f"/content/{MODEL_NAME}"
snapshot_download(repo_id=MODEL_ID, local_dir=local_path)
print(f"✅ 다운로드 완료: {local_path}")

In [ ]:
# 5. FP16 GGUF 변환
%cd /content/llama.cpp

!python convert_hf_to_gguf.py {local_path} \
    --outfile /content/{MODEL_NAME}-fp16.gguf \
    --outtype f16

print("✅ FP16 GGUF 변환 완료")

In [ ]:
# 6. 양자화
QUANTIZATIONS = ["Q8_0", "Q5_K_M", "Q4_K_M"]

for quant in QUANTIZATIONS:
    print(f"🔄 {quant} 양자화 중...")
    !./build/bin/llama-quantize \
        /content/{MODEL_NAME}-fp16.gguf \
        /content/{MODEL_NAME}-{quant}.gguf \
        {quant}
    print(f"✅ {quant} 완료")

In [ ]:
# 7. HuggingFace 업로드
from huggingface_hub import HfApi, create_repo

api = HfApi()
create_repo(OUTPUT_REPO, repo_type="model", exist_ok=True)

# 모든 GGUF 파일 업로드
import glob
for gguf_file in glob.glob(f"/content/{MODEL_NAME}*.gguf"):
    print(f"📤 업로드: {gguf_file}")
    api.upload_file(
        path_or_fileobj=gguf_file,
        path_in_repo=gguf_file.split("/")[-1],
        repo_id=OUTPUT_REPO,
        repo_type="model",
    )

print(f"✅ 업로드 완료: https://huggingface.co/{OUTPUT_REPO}")